In [1]:
import sys


sys.path.append("..")
from common_utils import set_data_home

set_data_home("~/datasets")
from common_utils import DATA_HOME, join
from lstm.sales_data import Sales_Dataset
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
SALE_HOME = join(DATA_HOME, "sales_data")
MODEL_HOME = join(DATA_HOME, "sale_model")

In [2]:
I, H, B = 72, 72, 6
HEAD = 6
num_fams = 33

sd = Sales_Dataset(SALE_HOME, seq_len=500, is_train=False, device=device)
# model.load_state_dict(torch.load("./sales_model_600.pth"))
sd[0][0].shape, sd[1][0].shape, sd[2][0].shape

(torch.Size([500, 72]), torch.Size([500, 72]), torch.Size([500, 72]))

In [3]:
from torch.nn import Transformer, Linear


class MyTransformer(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.trans = Transformer(d_model=I, nhead=HEAD, batch_first=True).cuda()
        self.linear = Linear(I, H)

    def forward(self, src, tgt):
        return self.linear(self.trans(src, tgt))

### log returns to sales

In [4]:
def ret_2_sale(data: torch.Tensor, base_sales: torch.Tensor):
    return 10**data * base_sales

### Perform Inference

In [15]:
import pandas as pd
from datetime import timedelta

INFERED_DAYS = 16

model = MyTransformer().cuda()
model.load_state_dict(torch.load("sales_model_5500.pth"))
sales = pd.read_csv(join(SALE_HOME, "test.csv"), index_col=0)
sales["sales"] = 0.0
base_sales = sd.base_sales.set_index(["store_nbr", "date"])

for X, store_id, lastday in sd:
    X = X.cuda()
    prev_sales = base_sales.loc[(store_id, lastday)].sales
    prev_sales = torch.tensor(prev_sales.to_numpy()).cuda()

    # infer and update input for each store
    for i in range(INFERED_DAYS):
        yhat = model(X, X[-1].unsqueeze(0))
        X = torch.concat((X[1:], yhat))

        # compute actual sales
        yhat_rets = yhat[0][:66][
            ::2
        ]  # once fetched first 66 (returns, promotions), get every other value, 33 in total
        prev_sales = ret_2_sale(yhat_rets, prev_sales)
        ts = (lastday + timedelta(days=i + 1)).strftime("%Y-%m-%d")

        # write to each family in the answer dataframe
        for j, f in enumerate(sd.families):
            sales.loc[
                (sales.date == ts)
                & (sales.store_nbr == store_id)
                & (sales.family == f),
                "sales",
            ] = (
                prev_sales[j].cpu().item()
            )

### output the answer

In [16]:
sales

,date,store_nbr,family,onpromotion,sales
id,,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0,4.475492
3000889,2017-08-16,1,BABY CARE,0,0.000000
3000890,2017-08-16,1,BEAUTY,2,5.498955
3000891,2017-08-16,1,BEVERAGES,20,1957.293646
3000892,2017-08-16,1,BOOKS,0,0.000000
...,...,...,...,...,...
3029395,2017-08-31,9,POULTRY,1,308.598288
3029396,2017-08-31,9,PREPARED FOODS,0,137.245985
3029397,2017-08-31,9,PRODUCE,1,2797.975151


In [17]:
sales.drop(columns=["store_nbr", "date", "family", "onpromotion"]).sort_values(
    "id"
).sort_index().to_csv("answer.csv", index=True)